In [1]:
import pandas as pd
import numpy as np
import openmatrix as omx
import matplotlib.pyplot as plt

### Link volume check

In [ ]:
# the volume files below are exported from the loaded network in EMME

volume_am = pd.read_csv(r"C:\abm_runs\alie\tasks\convergence\volume\AM.csv", delimiter='\t')
volume_pm = pd.read_csv(r"C:\abm_runs\alie\tasks\convergence\volume\PM.csv", delimiter='\t')

In [ ]:
link_type = {"freeway": [1], "Arterial": [2,3], "Collector":[4,5,6], "Local/Other": [7,8,9]}

volume_by_type = {}
for type, code in link_type.items():
    volume_by_type[type] = volume_pm[volume_pm.type.isin(code)&(volume_am.lanes>0)]

for type, _ in link_type.items():
       
    volume_by_type[type]['diff'] = volume_by_type[type]['@autovol_iter_2'] - volume_by_type[type]['@autovol_iter_3']
    volume_by_type[type]['diff_percent'] = volume_by_type[type]['diff'] / volume_by_type[type]['@autovol_iter_2']*100
    volume_by_type[type]['diff_percent'].replace([np.inf, -np.inf], 0, inplace=True)
    # do a bar plot
    # plt.figure()
    # plt.hist(volume_by_type[type]['diff_percent'], bins=100)
    # plt.title(type)
    # plt.xlabel('Difference in Volume (%)')
    # plt.ylabel('Number of Links')
    # plt.show()

    #compute and print share of links with abs difference in volume <5%
    print(type)
    print('Share of links with abs difference in volume <5%:', len(volume_by_type[type][abs(volume_by_type[type]['diff_percent'])<5])/len(volume_by_type[type]))
    print('Share of links with abs difference in volume <10%:', len(volume_by_type[type][(5<abs(volume_by_type[type]['diff_percent'])) & (abs(volume_by_type[type]['diff_percent'])<10)])/len(volume_by_type[type]))
    print('Share of links with abs difference in volume <25%:', len(volume_by_type[type][(10<abs(volume_by_type[type]['diff_percent'])) & (abs(volume_by_type[type]['diff_percent'])<25)])/len(volume_by_type[type]))
    print('Share of links with abs difference in volume >25%:', len(volume_by_type[type][abs(volume_by_type[type]['diff_percent'])>25])/len(volume_by_type[type]))
    print('-------------------------------------------------')



In [ ]:
results = {}
for type, _ in link_type.items():
       
    volume_by_type[type]['diff'] = volume_by_type[type]['@autovol_iter_3'] - volume_by_type[type]['@auto_volume']
    volume_by_type[type]['diff_percent'] = volume_by_type[type]['diff'] / volume_by_type[type]['@autovol_iter_3']*100
    volume_by_type[type]['diff_percent'].replace([np.inf, -np.inf], 0, inplace=True)
    # do a bar plot
    # plt.figure()
    # plt.hist(volume_by_type[type]['diff_percent'], bins=100)
    # plt.title(type)
    # plt.xlabel('Difference in Volume (%)')
    # plt.ylabel('Number of Links')
    # plt.show()

    #compute and print share of links with abs difference in volume <5%
    print(type)
    print('Share of links with abs difference in volume <5%:', len(volume_by_type[type][abs(volume_by_type[type]['diff_percent'])<5])/len(volume_by_type[type]))
    print('Share of links with abs difference in volume <10%:', len(volume_by_type[type][(5<abs(volume_by_type[type]['diff_percent'])) & (abs(volume_by_type[type]['diff_percent'])<10)])/len(volume_by_type[type]))
    print('Share of links with abs difference in volume <25%:', len(volume_by_type[type][(10<abs(volume_by_type[type]['diff_percent'])) & (abs(volume_by_type[type]['diff_percent'])<25)])/len(volume_by_type[type]))
    print('Share of links with abs difference in volume >25%:', len(volume_by_type[type][abs(volume_by_type[type]['diff_percent'])>25])/len(volume_by_type[type]))
    print('-------------------------------------------------')
    results[type] = {
    '<5%': len(volume_by_type[type][abs(volume_by_type[type]['diff_percent'])<5])/len(volume_by_type[type]),
    '5-10%': len(volume_by_type[type][(5<=abs(volume_by_type[type]['diff_percent'])) & (abs(volume_by_type[type]['diff_percent'])<10)])/len(volume_by_type[type]),
    '10-15%': len(volume_by_type[type][(10<=abs(volume_by_type[type]['diff_percent'])) & (abs(volume_by_type[type]['diff_percent'])<15)])/len(volume_by_type[type]),
    '15-25%':len(volume_by_type[type][(10<abs(volume_by_type[type]['diff_percent'])) & (abs(volume_by_type[type]['diff_percent'])<25)])/len(volume_by_type[type]),
    '>25%': len(volume_by_type[type][abs(volume_by_type[type]['diff_percent'])>=25])/len(volume_by_type[type])}



### Skim Check


In [ ]:
skim_am = {}
for iter in [1,2,3,4]:
    skim_am[iter] = {}
skim_pm = {}
for iter in [1,2,3,4]:
    skim_pm[iter] = {}


for iter in [1,2,3,4]:
    skim = omx.open_file(rf"C:\abm_runs\alie\output\ITERATIONS\skims_iter_{iter}\traffic_skims_AM.omx")
    skim_am[iter]['SOV_M_TIME'] = np.array(skim['SOV_NT_M_TIME__AM'])
    skim_am[iter]['HOV2_M_TIME'] = np.array(skim['HOV2_M_TIME__AM'])
    skim_am[iter]['HOV3_M_TIME'] = np.array(skim['HOV3_M_TIME__AM'])
    skim.close()
    skim = omx.open_file(rf"C:\abm_runs\alie\output\ITERATIONS\skims_iter_{iter}\traffic_skims_PM.omx")
    skim_pm[iter]['SOV_M_TIME'] = np.array(skim['SOV_NT_M_TIME__PM'])
    skim_pm[iter]['HOV2_M_TIME'] = np.array(skim['HOV2_M_TIME__PM'])
    skim_pm[iter]['HOV3_M_TIME'] = np.array(skim['HOV3_M_TIME__PM'])
    skim.close()


In [ ]:
# compute rmse between two numpy arrays
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

iter = 3
for core in ['SOV_M_TIME', 'HOV2_M_TIME', 'HOV3_M_TIME']:
    print(core)
    print(f"am: {rmse(skim_am[iter][core] , skim_am[iter-1][core])}")
    print(f"pm: {rmse(skim_pm[iter][core] , skim_pm[iter-1][core])}")
    print('----------------------')